# リッジ回帰の実装 (NumPy)

このノートブックでは、NumPyのみを使用してリッジ回帰をスクラッチから実装します。
リッジ回帰は、通常の最小二乗法にL2正則化項を加えることで、係数の大きさを抑制し、多重共線性問題に対処したり、モデルの過学習を防いだりする手法です。

## 1. ライブラリのインポート

In [6]:
import numpy as np

## 2. リッジ回帰の理論的背景

通常の線形回帰（最小二乗法）では、目的関数（損失関数）は残差平方和 (RSS) でした。
$$ L(\beta) = \sum_{i=1}^{n} (y_i - \mathbf{x}_i^T \beta)^2 = ||\mathbf{y} - \mathbf{X}\beta||^2_2 $$
ここで、$\mathbf{y}$ は目的変数のベクトル、$\mathbf{X}$ は計画行列（特徴量行列の先頭に1の列を追加したもの）、$\beta$ は回帰係数のベクトル（切片を含む）です。
この損失関数を最小化する $\beta$ の解は、正規方程式によって与えられます:
$$ \hat{\beta} = (\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T \mathbf{y} $$
しかし、この方法にはいくつかの問題点があります。
1.  **多重共線性**: 説明変数間に強い相関がある場合、$\mathbf{X}^T \mathbf{X}$ の逆行列が不安定になったり、計算できなくなったりします。
2.  **過学習**: モデルが訓練データに過剰に適合し、未知のデータに対する予測性能が低下することがあります。これは特に特徴量が多い場合に顕著です。

### リッジ回帰の目的関数
リッジ回帰は、これらの問題に対処するために、損失関数にL2正則化項（係数のL2ノルムの二乗）を追加します。
$$ L_{ridge}(\beta_0, \mathbf{\beta}_{rest}) = ||\mathbf{y} - (\mathbf{X}_{rest}\mathbf{\beta}_{rest} + \beta_0)||^2_2 + \alpha ||\mathbf{\beta}_{rest}||^2_2 $$
ここで、
*   $\beta_0$ は切片項です。
*   $\mathbf{\beta}_{rest}$ は切片以外の回帰係数のベクトルです $(\beta_1, \beta_2, ..., \beta_p)$。
*   $\mathbf{X}_{rest}$ は切片項に対応する列を除いた特徴量行列です。
*   $\alpha \ge 0$ は正則化パラメータ（またはハイパーパラメータ）で、ペナルティの強さを調整します。
*   $||\mathbf{\beta}_{rest}||^2_2 = \sum_{j=1}^{p} \beta_j^2$ です。
一般的に、切片 $\beta_0$ は正則化の対象外とします。これは、切片は特徴量のスケールに依存せず、予測値のベースラインを調整する役割を持つためです。

計画行列 $\mathbf{X}_b = [\mathbf{1} | \mathbf{X}_{rest}]$ と係数ベクトル $\mathbf{\beta} = [\beta_0 | \mathbf{\beta}_{rest}^T]^T$ を用いると、リッジ回帰の正規方程式は次のように表せます。
$$ \hat{\mathbf{\beta}}_{ridge} = (\mathbf{X}_b^T \mathbf{X}_b + \alpha \mathbf{I}^*)^{-1} \mathbf{X}_b^T \mathbf{y} $$
ここで、$\mathbf{I}^*$ は $(p+1) \times (p+1)$ の行列で、最初の対角要素 $(I^*_{00})$ が0で、残りの対角要素 $(I^*_{jj}, j=1,...,p)$ が1である行列です。これにより、切片 $\beta_0$ にはペナルティが課されません。

### $\alpha$ の効果
*   $\alpha = 0$: リッジ回帰は通常の最小二乗法と一致します（ただし、$\mathbf{X}^T \mathbf{X}$ が正則な場合）。
*   $\alpha > 0$: $(\mathbf{X}_b^T \mathbf{X}_b + \alpha \mathbf{I}^*)$ は正則になりやすく、逆行列が安定して計算できます。多重共線性がある場合でも解が得られやすくなります。
*   $\alpha$ が大きくなるほど、回帰係数 $\mathbf{\beta}_{rest}$ の絶対値は0に近づきます（シュリンケージ効果）。これにより、モデルの複雑さが抑えられ、過学習を防ぐ効果が期待できます。
適切な $\alpha$ の値は、交差検証などを用いて選択する必要があります。

## 3. データセットの準備

簡単なサンプルデータを作成してテストします。

*   `X_sample`: 2つの説明変数を持つ特徴量データ
*   `y_sample`: 目的変数

In [7]:
# サンプルデータ
# X は (サンプル数, 特徴量数) の形状
X_train_sample = np.array([
    [1, 1],
    [1, 2],
    [2, 2],
    [2, 3],
    [3, 2],
    [3, 4],
    [4, 4],
    [4, 5]
])
# y は (サンプル数,) の形状
# y = 2*X1 + 1*X2 + random noise
np.random.seed(42) # 再現性のためのシード設定
y_train_sample = 2 * X_train_sample[:, 0] + 1 * X_train_sample[:, 1] + np.random.normal(0, 0.5, X_train_sample.shape[0])

print("サンプル特徴量 X_train_sample (shape):", X_train_sample.shape)
print(X_train_sample)
print("\nサンプル目的変数 y_train_sample (shape):", y_train_sample.shape)
print(y_train_sample)

サンプル特徴量 X_train_sample (shape): (8, 2)
[[1 1]
 [1 2]
 [2 2]
 [2 3]
 [3 2]
 [3 4]
 [4 4]
 [4 5]]

サンプル目的変数 y_train_sample (shape): (8,)
[ 3.24835708  3.93086785  6.32384427  7.76151493  7.88292331  9.88293152
 12.78960641 13.38371736]


## 4. リッジ回帰モデルクラスの実装

`RidgeRegression` クラスを作成します。

### 4.1 概念説明

*   **`__init__` メソッド**:
    *   正則化パラメータ `alpha` を受け取り、インスタンス変数として保存します。
    *   回帰係数 `self.coef_` (β̂_rest) と切片 `self.intercept_` (β̂₀) を格納するための変数を初期化します。

*   **`fit` メソッド**:
    *   **目的**: 訓練データ `X_train` と `y_train`、および指定された `alpha` を用いて、最適な回帰係数と切片を計算します。
    *   **処理の流れ**:
        1.  **計画行列の作成**: 入力された特徴量行列 `X_train` の先頭に、切片項に対応する「1」のみからなる列を追加します。これにより、計画行列 `X_b` を作成します。
            `X_b = [1, X₁, X₂, ..., Xₚ]`
        2.  **正則化項の準備**: 切片項を正則化の対象外とするため、対角成分が `[0, alpha, alpha, ..., alpha]` となるようなペナルティ行列 `alpha_I_star` を作成します。具体的には、`(特徴量数 + 1) x (特徴量数 + 1)` の単位行列を作成し、その最初の対角要素 `(0,0)` を0に設定した後、`alpha` を乗じます。
        3.  **リッジ回帰の正規方程式の計算**: `β̂_ridge = (X_bᵀX_b + αI*)⁻¹X_bᵀY` の式に従って、回帰係数のベクトル `beta_hat` を計算します。
            *   `X_b.T`: `X_b` の転置
            *   `np.linalg.inv()`: 逆行列の計算
            *   `@` または `np.dot()`: 行列の積
            *   多重共線性により逆行列が計算できない場合に備えて、`np.linalg.pinv()` (擬似逆行列) をフォールバックとして使用します（ただし、`alpha > 0` のリッジ回帰では通常、`X_bᵀX_b + αI*` は正則になります）。
        4.  **係数の保存**: 計算された `beta_hat` の最初の要素を切片 `self.intercept_` (β̂₀) とし、残りの要素を回帰係数 `self.coef_` (β̂₁, ..., β̂ₚ) として保存します。

*   **`predict` メソッド**:
    *   **目的**: 学習済みのモデル（切片と回帰係数）を使って、新しい特徴量データ `X_test` に対する目的変数の値を予測します。
    *   **処理の流れ** (重回帰と同じ):
        1.  **計画行列の作成**: `fit` メソッドと同様に、`X_test` の先頭に切片項の「1」の列を追加して `X_b_test` を作成します。
        2.  **予測値の計算**: `Y_pred = X_b_test @ β̂_ridge` の式に従って予測値を計算します。ここで `β̂_ridge` は `fit` で学習した切片と回帰係数を結合したベクトルです。
            具体的には、`y_pred = X_test @ self.coef_ + self.intercept_` としても計算できます。

### 4.2 `RidgeRegression` クラスの実装

In [8]:
class RidgeRegression:
    def __init__(self, alpha=1.0):
        self.alpha = alpha          # 正則化パラメータ
        self.intercept_ = None      # 切片 (β₀)
        self.coef_ = None           # 回帰係数 (β₁, ..., βₚ)
        self._beta_hat = None       # 切片と回帰係数を含むベクトル [β₀, β₁, ..., βₚ]

    def fit(self, X, y):
        '''
        訓練データを用いてモデルを学習する
        Parameters:
            X(ndarray): 特徴量行列 (サンプル数, 特徴量数)
            y(ndarray): 目的変数ベクトル (サンプル数,)
        '''

        # 計画行列の作成 (切片項のために先頭に1の列を追加)
        n_samples, n_features = X.shape
        X_b = np.c_[np.ones((n_samples, 1)), X] # (n_samples, n_features + 1)

        # 正則化項のためのペナルティ行列 alpha * I* を作成
        # I* は (n_features + 1) x (n_features + 1) の行列で、
        # I*[0,0] = 0 (切片は正則化しない), I*[j,j] = 1 (j > 0)
        penalty_matrix = self.alpha * np.eye(n_features + 1)
        penalty_matrix[0, 0] = 0 # 切片項に対応するペナルティは0

        # 正規方程式を用いてβ_hatを計算
        # β_hat = (X_b.T @ X_b + alpha * I*)⁻¹ @ X_b.T @ y
        try:
            # (X_b.T @ X_b + penalty_matrix) の部分
            A = X_b.T @ X_b + penalty_matrix
            # 逆行列を計算して beta_hat を求める
            beta_hat = np.linalg.inv(A) @ X_b.T @ y
        except np.linalg.LinAlgError:
            # 逆行列が計算できない場合は擬似逆行列を使用
            # (alpha > 0 のリッジ回帰では通常発生しにくいが念のため)
            print("Warning: Singular matrix, using pseudo-inverse.")
            A = X_b.T @ X_b + penalty_matrix
            beta_hat = np.linalg.pinv(A) @ X_b.T @ y

        # 計算された係数を保存
        self._beta_hat = beta_hat
        self.intercept_ = beta_hat[0]
        self.coef_ = beta_hat[1:]

    def predict(self, X):
        '''
        学習済みモデルを用いて予測を行う
        Parameters:
            X(ndarray): 特徴量行列 (サンプル数, 特徴量数)
        Returns:
            ndarray: 予測値ベクトル (サンプル数,)
        '''

        if self.intercept_ is None or self.coef_ is None:
            raise ValueError("Model is not fitted yet. Call 'fit' before 'predict'.")
        
        # 計画行列の作成
        n_samples = X.shape[0]
        if X.ndim == 1: # 1サンプルの場合
             X_b_test = np.concatenate(([1], X))
        else:
             X_b_test = np.c_[np.ones((n_samples, 1)), X]

        # 予測値の計算
        y_pred = X_b_test @ self._beta_hat

        return y_pred

## 5. モデルの学習と予測 (サンプルデータ)

異なる `alpha` の値でリッジ回帰モデルを学習させ、係数の変化を確認します。

In [9]:
alphas_to_test = [0, 0.01, 0.1, 1, 10, 100]

print("サンプルデータでのリッジ回帰モデルの学習結果:\n")
for alpha_val in alphas_to_test:
    # モデルのインスタンス化
    ridge_model = RidgeRegression(alpha=alpha_val)
    
    # モデルの学習
    ridge_model.fit(X_train_sample, y_train_sample)
    
    print(f"Alpha = {alpha_val}")
    print(f"  切片 (β₀): {ridge_model.intercept_:.4f}")
    print(f"  回帰係数 (β₁, β₂): {ridge_model.coef_}")
    
    # alpha=1 のモデルで訓練データに対する予測もしてみる
    if alpha_val == 1:
        y_pred_train_sample = ridge_model.predict(X_train_sample)
        print("\n  訓練データに対する予測結果 (Alpha=1):")
        for i in range(len(y_train_sample)):
            print(f"    実測値: {y_train_sample[i]:.2f}, 予測値: {y_pred_train_sample[i]:.2f}, 誤差: {y_train_sample[i] - y_pred_train_sample[i]:.2f}")
    print("---")

# alpha=1 のモデルで新しいデータに対する予測
ridge_model_alpha1 = RidgeRegression(alpha=1)
ridge_model_alpha1.fit(X_train_sample, y_train_sample)

X_new_sample = np.array([
    [1, 3],  # X1=1, X2=3
    [3, 3],  # X1=3, X2=3
    [5, 1]   # X1=5, X2=1
])

y_pred_new_sample = ridge_model_alpha1.predict(X_new_sample)

print("\n新しいデータに対する予測結果 (Alpha=1):")
for i in range(X_new_sample.shape[0]):
    print(f"  入力 X = {X_new_sample[i]}, 予測 Y = {y_pred_new_sample[i]:.2f}")

サンプルデータでのリッジ回帰モデルの学習結果:

Alpha = 0
  切片 (β₀): 0.0298
  回帰係数 (β₁, β₂): [2.0112685 1.0756538]
---
Alpha = 0.01
  切片 (β₀): 0.0337
  回帰係数 (β₁, β₂): [2.00721558 1.07780717]
---
Alpha = 0.1
  切片 (β₀): 0.0690
  回帰係数 (β₁, β₂): [1.97235068 1.09585834]
---
Alpha = 1
  切片 (β₀): 0.4034
  回帰係数 (β₁, β₂): [1.72769913 1.19228481]

  訓練データに対する予測結果 (Alpha=1):
    実測値: 3.25, 予測値: 3.32, 誤差: -0.08
    実測値: 3.93, 予測値: 4.52, 誤差: -0.58
    実測値: 6.32, 予測値: 6.24, 誤差: 0.08
    実測値: 7.76, 予測値: 7.44, 誤差: 0.33
    実測値: 7.88, 予測値: 7.97, 誤差: -0.09
    実測値: 9.88, 予測値: 10.36, 誤差: -0.47
    実測値: 12.79, 予測値: 12.08, 誤差: 0.71
    実測値: 13.38, 予測値: 13.28, 誤差: 0.11
---
Alpha = 10
  切片 (β₀): 2.6544
  回帰係数 (β₁, β₂): [1.0367537  1.01013916]
---
Alpha = 100
  切片 (β₀): 6.7412
  回帰係数 (β₁, β₂): [0.25235871 0.27073033]
---

新しいデータに対する予測結果 (Alpha=1):
  入力 X = [1 3], 予測 Y = 5.71
  入力 X = [3 3], 予測 Y = 9.16
  入力 X = [5 1], 予測 Y = 10.23


## 6. より実践的なデータセットでの利用と評価

scikit-learnのDiabetesデータセットを使用して、モデルの性能を評価します。

In [10]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target

# 特徴量の標準化 (リッジ回帰では特に重要)
# 標準化により、各特徴量が同等のスケールでペナルティを受けるようにするため
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [11]:
alphas_practical = [0.01, 0.1, 1.0, 10.0, 100.0]

print("\n実践的なデータセット (Diabetes) での評価:\n")
for alpha_val in alphas_practical:
    model_practical = RidgeRegression(alpha=alpha_val)
    model_practical.fit(X_train, y_train) # 標準化されたデータで学習

    y_pred_test = model_practical.predict(X_test)

    # 評価
    mse = mean_squared_error(y_test, y_pred_test)
    r2 = r2_score(y_test, y_pred_test)

    print(f"Alpha = {alpha_val}")
    print(f"  切片: {model_practical.intercept_:.4f}")
    # print(f"  係数 (最初の5つ): {model_practical.coef_[:5]}") # 全部は長いので一部表示
    print(f"  平均二乗誤差 (MSE): {mse:.4f}")
    print(f"  決定係数 (R²): {r2:.4f}")
    print("---")


実践的なデータセット (Diabetes) での評価:

Alpha = 0.01
  切片: 151.3455
  平均二乗誤差 (MSE): 2900.0755
  決定係数 (R²): 0.4526
---
Alpha = 0.1
  切片: 151.3447
  平均二乗誤差 (MSE): 2899.0581
  決定係数 (R²): 0.4528
---
Alpha = 1.0
  切片: 151.3390
  平均二乗誤差 (MSE): 2892.0301
  決定係数 (R²): 0.4541
---
Alpha = 10.0
  切片: 151.3491
  平均二乗誤差 (MSE): 2875.6676
  決定係数 (R²): 0.4572
---
Alpha = 100.0
  切片: 151.6225
  平均二乗誤差 (MSE): 2857.3610
  決定係数 (R²): 0.4607
---


## 7. 考察

*   このスクラッチ実装では、リッジ回帰の正規方程式 `β̂_ridge = (X_bᵀX_b + αI*)⁻¹X_bᵀY` をNumPyを用いて直接計算しました。切片項は正則化の対象外とする一般的なアプローチを採用しました。
*   **長所**:
    *   アルゴリズムの内部動作、特に正則化項がどのように係数推定に影響を与えるかを理解しやすい。
    *   `alpha > 0` の場合、$(\mathbf{X}_b^T \mathbf{X}_b + \alpha \mathbf{I}^*)$ が正則になりやすいため、多重共線性があるデータに対しても安定して係数を推定できる可能性があります。
    *   係数の大きさを抑制することで、モデルの過学習を軽減する効果が期待できます。

*   **短所・注意点**:
    *   **ハイパーパラメータ `alpha` の選択**: リッジ回帰の性能は `alpha` の値に大きく依存します。適切な `alpha` を見つけるためには、交差検証 (Cross-Validation) などの手法が必要です。このノートブックでは手動でいくつかの値を試しましたが、実践ではより体系的な探索が求められます。
    *   **計算コスト**: 特徴量数が非常に多い場合、`X_bᵀX_b` の計算やその逆行列計算は依然としてコストが高い可能性があります (主に逆行列計算が `O(p³)`、pは特徴量数)。scikit-learnなどのライブラリは、より数値的に安定し、大規模データにも対応できるソルバー（例：共役勾配法、特異値分解(SVD)ベースの方法、座標降下法など）を内部で使用している場合があります。
    *   **特徴量のスケーリング**: リッジ回帰では、ペナルティ項 $$\alpha ||\mathbf{\beta}_{rest}||^2_2$$ が各係数 $\beta_j$ の大きさに依存するため、特徴量のスケールが異なると、スケールの大きな特徴量に対応する係数が不当に小さくされてしまう可能性があります。そのため、事前に特徴量を標準化（StandardScalerなど）または正規化（MinMaxScalerなど）することが強く推奨されます。今回の実践的なデータセットの例では標準化を行いました。

*   **scikit-learnとの比較**: scikit-learnの `sklearn.linear_model.Ridge` は、より高度なソルバーオプション（'svd', 'cholesky', 'sparse_cg', 'lsqr', 'sag', 'saga'など）を提供し、大規模データやさまざまな状況に対応できるように最適化されています。また、`RidgeCV` のように交差検証による `alpha` の自動選択機能も備えています。本スクラッチ実装は、基本的な概念を理解するためのものと位置づけられます。